# Chapter 1 — Classifying Linearly Separable Data with Neural Nets
## Introduction
Your journey to understand Neural Networks starts with a humble example of building a binary classifier for linearly separable data. 'Linearly seperable data'  means that a line can shatter (or classify) the data into two classes. For two-dimensional data, the decision boundary is a line, and for three or more dimensions in the data, the decision boundary would be a plane and a milti-dimensional daat the decision boundary would be a plane.

## The Dataset
We’ll use the example below to illustrate the workings at the Neural Network. Imagine you are running a computing infrastructure where you have just two tiers of pricing, VIP and Regular. You want to classify each user into one of the two tiers (or classes) based on the two resources they consume. Let's call these resources x1 and x2 (for instance, the resources could be the number of CPU cores and the network bandwidth). Also, you are just starting out providing such a service and you have a only handful of users (i.e., a small number of data samples). Your task is to draw the decision boundary between the two classes of users. The decision boundary is a straight line and you are granted the assumption that the data is linearly separable.

![startup pic](plots/first-diag.png)




A common practice is to center the data around the origin (0, 0) by taking the mean and subtracting each data point from the mean.

![](plots/centered-data.png)